## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://pypi.org/project/rank-bm25/

In [217]:
path = '../04-filtrage/output/'
acteur = 'pinel'
tag = ''

if tag:
    csv_file = acteur + '_' + tag + '_significant-collocations.csv'

else:
    csv_file = acteur + '_significant-collocations.csv'

### **Lire le vocabulaire** (termes retenus au prétraitement)

In [218]:
from pandas import *

with open(path+csv_file, encoding='utf-8') as f:
    csv = read_csv(f).drop(columns = ['Unnamed: 0'])
    # On va traiter les apostrophes et les parenthèses avec le underscore ("_") ; le tokenizer de sklearn ne les aime pas 
    csv['Terme formatté'] = csv["Terme"].apply(lambda x: str(x).replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_"))

csv

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté
0,pinel,institut,NOM,568,108,-,False,False,institut
1,pinel,psychiatrie,NOM,476,99,-,False,True,psychiatrie
2,pinel,psychiatrie légale,NOM ADJ,416,94,-inf,True,False,psychiatrie légale
3,pinel,candidature,NOM,285,71,-,False,False,candidature
4,pinel,philippe-pinel,NOM,233,94,-,False,False,philippe_pinel
...,...,...,...,...,...,...,...,...,...
432,pinel,programmes de traitement,NOM PRP NOM,16,5,200.17365265270783,False,False,programmes de traitement
433,pinel,statut liste de rappel,NOM NOM PRP NOM,16,10,262.04235005258175,False,False,statut liste de rappel
434,pinel,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,2,190.60835307163848,False,False,temps complet permanent titre du poste
435,pinel,dotation,NOM,16,6,-,False,False,dotation


In [219]:
vocabulaire = [t.lower() for t in list(csv['Terme formatté'])]

In [220]:
print('On a un vocabulaire de {} formes.'.format(len(vocabulaire)))

On a un vocabulaire de 437 formes.


In [221]:
vocabulaire

['institut',
 'psychiatrie',
 'psychiatrie légale',
 'candidature',
 'philippe_pinel',
 'poste',
 'santé',
 'soins',
 'institut national',
 'psychiatrie légale philippe_pinel',
 'institut national de psychiatrie',
 'institut national de psychiatrie légale',
 'formation',
 'institut national de psychiatrie légale philippe_pinel',
 'services',
 'titre',
 'accès',
 'employés',
 'développement',
 'temps',
 'gestion',
 'jours',
 'évaluation',
 'congé',
 'statut',
 'avantages',
 'sécurité',
 'minorités',
 'gamme',
 'besoins',
 'travail',
 'clinique',
 'sélection',
 'établissement',
 'programme',
 'pratiques',
 'milieu',
 'patients',
 'processus',
 'qualité',
 'projets',
 'environnement',
 'justice',
 'compétences',
 'vie',
 'formations',
 'professionnels',
 'amélioration',
 'temps complet',
 'capacité',
 'professionnelle',
 'déploiement',
 'jour',
 'santé mentale',
 'projet',
 'amélioration continue',
 'service',
 'statut temps',
 'conseiller',
 'utilisation',
 'recherche',
 'clientèle',
 'a

### **Lire le corpus**

In [222]:
import os, shutil, re
from pathlib import Path
from os import path
from pandas import *

base_path = '../03-corpus/2-data/1-fr/'
if tag:
    base_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else:
    base_path = path.join(base_path, acteur +  '.csv')
        
with open(base_path, "r", encoding = "UTF-8") as f:
    data = read_csv(base_path, sep=',')
    text = data['text'].tolist()

In [223]:
text = text[:round(len(text))]

nb_docs = len(text)

print("On a donc un corpus de {} documents.".format(nb_docs))

On a donc un corpus de 115 documents.


### **Nettoyage**

In [224]:
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    
punct = '[!#$%&•►*+,;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [re.sub(punct, ' ', t).replace("' ", "'" ) for t in corpus]
corpus = [re.sub(phones, ' STOP ', t) for t in corpus]
corpus = [re.sub(postals, ' STOP ', t) for t in corpus]
corpus = [t.replace("  ", " " ) for t in corpus]
corpus = [t.replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_") for t in corpus]


On va commencer par utiliser le CountVectorizer pour valider que l'implémentation de sklearn arrive bien au même compte que nous 

In [225]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(vocabulary=vocabulaire, ngram_range=(1,10), token_pattern='\w+')
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

features_names = vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(X.T.todense(), index=features_names, columns=corpus_index).transpose()

In [226]:
csv['TF (sklearn)'] = 0
for t in vocabulaire:
    csv.loc[csv['Terme formatté'] == t, 'TF (sklearn)'] = df[t].sum()


csv['DF (sklearn)'] = 0
for t in vocabulaire:
    freqdoc = len(df[df[t] != 0])
    csv.loc[csv['Terme formatté'] == t, 'DF (sklearn)'] = freqdoc
csv

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn)
0,pinel,institut,NOM,568,108,-,False,False,institut,3,3
1,pinel,psychiatrie,NOM,476,99,-,False,True,psychiatrie,474,99
2,pinel,psychiatrie légale,NOM ADJ,416,94,-inf,True,False,psychiatrie légale,412,94
3,pinel,candidature,NOM,285,71,-,False,False,candidature,285,71
4,pinel,philippe-pinel,NOM,233,94,-,False,False,philippe_pinel,233,94
...,...,...,...,...,...,...,...,...,...,...,...
432,pinel,programmes de traitement,NOM PRP NOM,16,5,200.17365265270783,False,False,programmes de traitement,16,5
433,pinel,statut liste de rappel,NOM NOM PRP NOM,16,10,262.04235005258175,False,False,statut liste de rappel,16,10
434,pinel,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,2,190.60835307163848,False,False,temps complet permanent titre du poste,16,2
435,pinel,dotation,NOM,16,6,-,False,False,dotation,16,6


In [227]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import RegexpTokenizer

#toknizer = RegexpTokenizer()

# max_df : ignore words that appear in 85% of documents, 
# min df:  ignore words that appear in less than 1% of documents 
# vocabulary = vocabulaire

# Sans utiliser le vocabulaire
# tfidf = TfidfVectorizer(min_df=0.1, stop_words=None, ngram_range=(2,4), max_df=0.85, use_idf=True)

# vocabulary = vocabulaire
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulaire, ngram_range=(1,10), lowercase=False, token_pattern='\w+')
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [228]:
features_names = tfidf_vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfidf.T.todense(), index=features_names, columns=corpus_index).transpose()

In [229]:
terms_tfidf = {term: df[term].max() for term in df}
zeros = {term : value for term, value in terms_tfidf.items() if terms_tfidf[term] == 0}

In [230]:
zeros

{'acquisition': 0.0,
 'urgence': 0.0,
 'excellence': 0.0,
 'amélioration continue des pratiques': 0.0,
 'amélioration continue des pratiques cliniques': 0.0,
 'accès à l_égalité': 0.0,
 'établissement de pointe': 0.0,
 'égalité': 0.0,
 'échéance': 0.0,
 'employé': 0.0,
 'avantages sociaux bonifiés': 0.0,
 'élaboration': 0.0,
 'évaluation du risque': 0.0,
 'évaluation du risque de violence': 0.0,
 'entretien': 0.0,
 'urgence et sécurité civile': 0.0,
 'urgence et sécurité': 0.0,
 'abus': 0.0,
 'institut philippe_pinel': 0.0,
 'ordre des psychologues': 0.0,
 'ordre des psychologues du québec': 0.0,
 'enseignement universitaire': 0.0}

In [231]:
len(zeros) / len(vocabulaire) *100

5.034324942791762

In [232]:
terms_tfidf

{'institut': 0.1370182411298561,
 'psychiatrie': 0.30069222602243645,
 'psychiatrie légale': 0.22739995485758455,
 'candidature': 0.24415595092662587,
 'philippe_pinel': 0.4178085156211113,
 'poste': 0.19911299440861807,
 'santé': 0.27883592993595585,
 'soins': 0.37983705417145996,
 'institut national': 0.1370182411298561,
 'psychiatrie légale philippe_pinel': 0.18898600794747505,
 'institut national de psychiatrie': 0.1370182411298561,
 'institut national de psychiatrie légale': 0.1370182411298561,
 'formation': 0.1800524491118031,
 'institut national de psychiatrie légale philippe_pinel': 0.1370182411298561,
 'services': 0.2879346614731307,
 'titre': 0.19027801346659381,
 'accès': 0.14247696195026935,
 'employés': 0.21985593657959956,
 'développement': 0.32844316940081075,
 'temps': 0.21549915190722227,
 'gestion': 0.2669162295351783,
 'jours': 0.15100708484467396,
 'évaluation': 0.36376553067228157,
 'congé': 0.20334317860858284,
 'statut': 0.21108649734276433,
 'avantages': 0.13618

In [233]:
terms_weighted = DataFrame(terms_tfidf.items(), columns=['Terme formatté', 'TF-IDF'])

In [234]:
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

terms_weighted = terms_weighted.drop_duplicates(keep='first')
terms_weighted

,Terme formatté,TF-IDF
225,horaire,0.971378
212,télésanté,0.837787
417,distance,0.767332
428,cognitions,0.739558
216,centre,0.730198
...,...,...
419,ordre des psychologues,0.000000
418,institut philippe_pinel,0.000000
388,entretien,0.000000
390,urgence et sécurité civile,0.000000


In [235]:
terms_weighted = pd.merge(csv, terms_weighted, on='Terme formatté').drop_duplicates(
  subset = ['Terme', 'Fréquence (TF)'],
  keep = 'first').reset_index(drop = True)

terms_weighted

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,pinel,institut,NOM,568,108,-,False,False,institut,3,3,0.137018
1,pinel,psychiatrie,NOM,476,99,-,False,True,psychiatrie,474,99,0.300692
2,pinel,psychiatrie légale,NOM ADJ,416,94,-inf,True,False,psychiatrie légale,412,94,0.227400
3,pinel,candidature,NOM,285,71,-,False,False,candidature,285,71,0.244156
4,pinel,philippe-pinel,NOM,233,94,-,False,False,philippe_pinel,233,94,0.417809
...,...,...,...,...,...,...,...,...,...,...,...,...
432,pinel,programmes de traitement,NOM PRP NOM,16,5,200.17365265270783,False,False,programmes de traitement,16,5,0.358892
433,pinel,statut liste de rappel,NOM NOM PRP NOM,16,10,262.04235005258175,False,False,statut liste de rappel,16,10,0.138686
434,pinel,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,2,190.60835307163848,False,False,temps complet permanent titre du poste,16,2,0.140975
435,pinel,dotation,NOM,16,6,-,False,False,dotation,16,6,0.444461


## **OKapi BM25**
https://hal.archives-ouvertes.fr/hal-00760158 

In [236]:
from rank_bm25 import BM25Okapi

In [237]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("(\w+\'|\w+-\w+|\(|\)|\w+)")

bm25 = BM25Okapi([tokenizer.tokenize(t) for t in corpus])

In [238]:
vocabulaire = csv["Terme"].dropna().tolist()

vocabulaire

['institut',
 'psychiatrie',
 'psychiatrie légale',
 'candidature',
 'philippe-pinel',
 'poste',
 'santé',
 'soins',
 'institut national',
 'psychiatrie légale philippe-pinel',
 'institut national de psychiatrie',
 'institut national de psychiatrie légale',
 'formation',
 'institut national de psychiatrie légale philippe-pinel',
 'services',
 'titre',
 'accès',
 'employés',
 'développement',
 'temps',
 'gestion',
 'jours',
 'évaluation',
 'congé',
 'statut',
 'avantages',
 'sécurité',
 'minorités',
 'gamme',
 'besoins',
 'travail',
 'clinique',
 'sélection',
 'établissement',
 'programme',
 'pratiques',
 'milieu',
 'patients',
 'processus',
 'qualité',
 'projets',
 'environnement',
 'justice',
 'compétences',
 'vie',
 'formations',
 'professionnels',
 'amélioration',
 'temps complet',
 'capacité',
 'professionnelle',
 'déploiement',
 'jour',
 'santé mentale',
 'projet',
 'amélioration continue',
 'service',
 'statut temps',
 'conseiller',
 'utilisation',
 'recherche',
 'clientèle',
 'a

In [239]:
#tokenizer = RegexpTokenizer(r"\w\'|\w+")
## Revoir ça
tokenized_queries = [tokenizer.tokenize(str(t)) for t in vocabulaire]

features_names = [t for t in set(vocabulaire)]
corpus_index = [corpus.index(n) for n in corpus]

tab = [bm25.get_scores(query) for query in tokenized_queries]
df = pd.DataFrame(tab, index=features_names, columns=corpus_index).transpose()

In [240]:
print(len(tokenized_queries))

tokenized_queries

437


[['institut'],
 ['psychiatrie'],
 ['psychiatrie', 'légale'],
 ['candidature'],
 ['philippe-pinel'],
 ['poste'],
 ['santé'],
 ['soins'],
 ['institut', 'national'],
 ['psychiatrie', 'légale', 'philippe-pinel'],
 ['institut', 'national', 'de', 'psychiatrie'],
 ['institut', 'national', 'de', 'psychiatrie', 'légale'],
 ['formation'],
 ['institut', 'national', 'de', 'psychiatrie', 'légale', 'philippe-pinel'],
 ['services'],
 ['titre'],
 ['accès'],
 ['employés'],
 ['développement'],
 ['temps'],
 ['gestion'],
 ['jours'],
 ['évaluation'],
 ['congé'],
 ['statut'],
 ['avantages'],
 ['sécurité'],
 ['minorités'],
 ['gamme'],
 ['besoins'],
 ['travail'],
 ['clinique'],
 ['sélection'],
 ['établissement'],
 ['programme'],
 ['pratiques'],
 ['milieu'],
 ['patients'],
 ['processus'],
 ['qualité'],
 ['projets'],
 ['environnement'],
 ['justice'],
 ['compétences'],
 ['vie'],
 ['formations'],
 ['professionnels'],
 ['amélioration'],
 ['temps', 'complet'],
 ['capacité'],
 ['professionnelle'],
 ['déploiement'],


In [241]:
df

,urgence,égalité,neuropsychologie,ateliers chercheurs,statut,demande,sens,amélioration continue des pratiques,processus de sélection,justice,...,titre d'établissement,ateliers,ans,statut temps complet,pinel,statut temps complet permanent titre,invite,pointe,enfants,soins
0,0.000000,1.089409,2.178818,0.000000,0.0,0.000000,0.000000,0.000000,1.089409,2.178818,...,0.000000,0.0,0.0,0.000000,0.000000,1.878900,1.878900,1.089409,0.0,0.000000
0,0.000000,1.089409,2.178818,0.000000,0.0,0.000000,0.000000,0.000000,1.089409,2.178818,...,0.000000,0.0,0.0,0.000000,0.000000,1.878900,1.878900,1.089409,0.0,0.000000
2,3.677196,1.557939,3.115878,0.000000,0.0,0.000000,0.000000,0.839867,5.235135,3.115878,...,2.688788,0.0,0.0,1.179704,1.179704,2.923267,1.923372,2.658616,0.0,0.000000
3,0.000000,1.483023,2.966045,1.483023,0.0,0.845190,0.845190,1.368290,1.184945,2.966045,...,0.000000,0.0,0.0,1.690380,0.845190,1.920010,10.256924,2.875324,0.0,0.000000
4,0.000000,1.476864,2.953729,1.476864,0.0,0.837231,0.837231,1.361307,1.177101,2.953729,...,0.000000,0.0,0.0,2.566344,1.729112,1.910885,7.733357,3.743445,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0.000000,0.777447,1.894180,0.000000,0.0,0.000000,1.306840,1.770544,0.000000,1.894180,...,0.000000,0.0,0.0,0.000000,0.000000,2.933038,1.921939,1.428423,0.0,1.269839
111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.339677,1.598562,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,3.246138,1.930100,1.339677,0.0,0.000000
111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.339677,1.598562,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,3.246138,1.930100,1.339677,0.0,0.000000
113,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.650491,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,3.205949,1.896936,1.099513,0.0,0.000000


In [242]:
#df.to_csv(base_path + titre + '_matrice-OkapiBM25.csv') # Si on veut avoir la matrice (mais le fichier peut être très volumineux)

In [243]:
terms_okapi = {term: df[term].max() for term in df}

In [244]:
terms_okapi

{'urgence': 3.6771958183635958,
 'égalité': 1.7153691265331956,
 'neuropsychologie': 3.3593119875491944,
 'ateliers chercheurs': 1.577478416248703,
 'statut': 0.0,
 'demande': 1.839440687407741,
 'sens': 1.673772917700278,
 'amélioration continue des pratiques': 1.7705443273577945,
 'processus de sélection': 5.235134801141695,
 'justice': 3.3593119875491944,
 'tâches': 8.716445719149085,
 'leadership': 10.274384701927184,
 'jours': 1.6642363198583643,
 'qualité': 10.274384701927184,
 'ordre': 1.7086220491068507,
 'abus': 1.828208438779628,
 'restrictions': 1.3509647138558487,
 'environnement': 1.5195141443238902,
 'candidature via le formulaire ci-dessous': 1.683002076446909,
 'communication': 1.828208438779628,
 'cours': 0.60300290352132,
 'tests': 1.3509647138558487,
 'problèmes': 3.330544786999705,
 'recommandations': 1.4677221768691364,
 'prime de milieu': 1.8321448509602845,
 'perceptions habituelles': 1.339225956200799,
 'respect': 1.8273681050484833,
 'évaluation du risque': 1.2

In [245]:
tab = DataFrame(terms_okapi.items(), columns=['Terme', 'OkapiBM25'])
tab

,Terme,OkapiBM25
0,urgence,3.677196
1,égalité,1.715369
2,neuropsychologie,3.359312
3,ateliers chercheurs,1.577478
4,statut,0.000000
...,...,...
432,statut temps complet permanent titre,5.562789
433,invite,10.256924
434,pointe,10.098992
435,enfants,5.366259


In [246]:
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

tab

,Terme,OkapiBM25
376,semaines,11.698948
218,fonctions,11.333153
303,internes,10.486546
11,leadership,10.274385
13,qualité,10.274385
...,...,...
410,événement,0.000000
101,amélioration continue des pratiques cliniques,0.000000
107,acquisition,0.000000
388,consultation,0.000000


In [247]:
terms_weighted

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,pinel,institut,NOM,568,108,-,False,False,institut,3,3,0.137018
1,pinel,psychiatrie,NOM,476,99,-,False,True,psychiatrie,474,99,0.300692
2,pinel,psychiatrie légale,NOM ADJ,416,94,-inf,True,False,psychiatrie légale,412,94,0.227400
3,pinel,candidature,NOM,285,71,-,False,False,candidature,285,71,0.244156
4,pinel,philippe-pinel,NOM,233,94,-,False,False,philippe_pinel,233,94,0.417809
...,...,...,...,...,...,...,...,...,...,...,...,...
432,pinel,programmes de traitement,NOM PRP NOM,16,5,200.17365265270783,False,False,programmes de traitement,16,5,0.358892
433,pinel,statut liste de rappel,NOM NOM PRP NOM,16,10,262.04235005258175,False,False,statut liste de rappel,16,10,0.138686
434,pinel,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,2,190.60835307163848,False,False,temps complet permanent titre du poste,16,2,0.140975
435,pinel,dotation,NOM,16,6,-,False,False,dotation,16,6,0.444461


In [248]:
tab = pd.merge(terms_weighted, tab, on="Terme")
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

In [249]:
base_path = '../05-transformation/'

In [250]:
if tag:
    file_path = base_path + acteur + '_' + tag + '_weighting_OKapiBM25.csv'

else: 
    file_path = base_path + acteur  + '_weighting_OKapiBM25.csv'
tab.to_csv(file_path)

In [251]:
tab

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF,OkapiBM25
72,pinel,semaines,NOM,78,62,-,False,False,semaines,78,62,0.128277,11.698948
279,pinel,fonctions,NOM,29,20,-,False,False,fonctions,29,20,0.311216,11.333153
315,pinel,internes,NOM,25,15,-,False,False,internes,19,9,0.096780,10.486546
399,pinel,leadership,NOM,17,17,-,False,False,leadership,17,17,0.100318,10.274385
39,pinel,qualité,NOM,111,73,-,False,False,qualité,111,73,0.136561,10.274385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,pinel,statut temps complet permanent salaire,NOM NOM ADJ ADJ NOM,19,19,223.77256855687267,False,False,statut temps complet permanent salaire,19,19,0.083621,0.000000
79,pinel,acquisition,NOM,75,70,-,False,False,acquisition,0,0,0.000000,0.000000
4,pinel,philippe-pinel,NOM,233,94,-,False,False,philippe_pinel,233,94,0.417809,0.000000
24,pinel,statut,NOM,138,66,-,False,False,statut,138,66,0.211086,0.000000
